# Example run

In [ ]:
# Import the module
import experiment

# Use the classes and functions
games = [experiment.GameOfChicken]  # Add your game classes

experiment_runner = experiment.ExperimentRunner(
    games=games,
    repetitions=2,  # Define the number of repetitions for each experiment
    results_dir='results/results_0',  # Directory to save the results => Loop can be made to save 
    analysis_dir='results/analysis_0',  # Directory to save the analysis
    rounds_per_experiment=2000,  # Number of rounds for each experiment
    base_lr=0.02,  # Set your desired base learning rate
    batch_size=128,  # Set your desired batch size
    decay_rate=0.95  # Set your desired decay rate
)

# Run the experiments
experiment_runner.run_experiments()

# Data aggregation

In [ ]:
import os
import json
import gzip
import numpy as np
import pandas as pd
import logging
from joblib import Parallel, delayed
import platform
import experiment

# Logging configuration
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Configuration
metric_titles = {
    'actor_losses_a': 'Actor Loss A',
    'critic_losses_a': 'Critic Loss A',
    'actor_losses_b': 'Actor Loss B',
    'critic_losses_b': 'Critic Loss B',
    'actions_a': 'Actions A',
    'actions_b': 'Actions B',
    'policy_history_a': 'Policy History A',
    'policy_history_b': 'Policy History B',
    'rewards_a': 'Rewards A',
    'rewards_b': 'Rewards B',
    # Add other metrics as needed
}

def load_results(file):
    """Load results from a single file."""
    try:
        with gzip.open(file, 'rt', encoding='UTF-8') as f:
            return json.load(f)
    except Exception as e:
        logger.error(f"Error reading {file}: {e}")
        return None

def flatten_list(data):
    """Flatten a nested list without recursion."""
    stack = [data]
    flattened = []
    while stack:
        current = stack.pop()
        if isinstance(current, list):
            stack.extend(reversed(current))
        else:
            flattened.append(current)
    return flattened

def flatten_policy_data(policy_list, continuous):
    """Flatten policy data while preserving innermost policy lists."""
    stack = [(policy_list, 0)]
    flattened_list = []
    while stack:
        current, depth = stack.pop()
        if isinstance(current, list) and depth < 2:
            stack.extend((item, depth + 1) for item in reversed(current))
        else:
            flattened_list.append(current if not continuous else [current[0][0], current[1][0]])
    return flattened_list

def precompute_rewards(game_class, action_labels):
    game_instance = game_class()
    rewards = {}
    for action_a in action_labels:
        for action_b in action_labels:
            reward_a, reward_b, _, _, _ = game_instance.step(action_a, action_b)
            rewards[(action_a, action_b)] = (reward_a, reward_b)
    return rewards

def calculate_rewards_vectorized(actions_a, actions_b, precomputed_rewards, continuous_actions, game_class):
    if continuous_actions:
        game_instance = game_class()
        rewards = np.array([game_instance.step(action_a, action_b)[:2] for action_a, action_b in zip(actions_a, actions_b)])
        return rewards[:, 0], rewards[:, 1]
    else:
        rewards = np.array([precomputed_rewards[(action_a, action_b)] for action_a, action_b in zip(actions_a, actions_b)])
        return rewards[:, 0], rewards[:, 1]

def save_dataframe(df, results_dir, metric, game):
    df['Game'] = game
    df['Transparency A'] = df['Transparency A'].astype('category')
    df['Transparency B'] = df['Transparency B'].astype('category')
    df['Game'] = df['Game'].astype('category')
    file_path = os.path.join(results_dir, f"{metric}.parquet")
    df.to_parquet(file_path, index=False, engine="pyarrow")

def load_dataframe(metric, results_dir):
    file_path = os.path.join(results_dir, f"{metric}.parquet")
    if os.path.exists(file_path):
        return pd.read_parquet(file_path, engine="pyarrow")
    return None

def process_file(file, continuous_games, metric, action_labels_dict):
    data = load_results(file)
    if data is None:
        return None

    seed = data.get('seed')
    transparency_a = data.get('transparency_a')
    transparency_b = data.get('transparency_b')
    game = data.get('game')
    continuous = (game in continuous_games)

    if 'policy' in metric:
        values = flatten_policy_data(data.get(metric, []), continuous)
    else:
        values = flatten_list(data.get(metric, []))

    step_data = {
        'Seed': seed,
        'Step': list(range(len(values))),
        'Transparency A': [transparency_a] * len(values),
        'Transparency B': [transparency_b] * len(values),
        metric_titles[metric]: values,
        'Game': [game] * len(values)
    }

    df = pd.DataFrame(step_data)

    if metric in ['actions_a', 'actions_b']:
        actions_a = flatten_list(data.get('actions_a', []))
        actions_b = flatten_list(data.get('actions_b', []))
        game_class = getattr(experiment, game)
        if game in continuous_games:
            precomputed_rewards = None
        else:
            action_labels = list(range(len(action_labels_dict[game])))
            precomputed_rewards = precompute_rewards(game_class, action_labels)

        rewards_a, rewards_b = calculate_rewards_vectorized(actions_a, actions_b, precomputed_rewards, continuous, game_class)
        rewards_df_a = pd.DataFrame({
            'Seed': seed,
            'Step': list(range(len(rewards_a))),
            'Rewards A': rewards_a,
            'Transparency A': [transparency_a] * len(rewards_a),
            'Transparency B': [transparency_b] * len(rewards_a),
            'Game': [game] * len(rewards_a)
        })
        rewards_df_b = pd.DataFrame({
            'Seed': seed,
            'Step': list(range(len(rewards_b))),
            'Rewards B': rewards_b,
            'Transparency A': [transparency_a] * len(rewards_b),
            'Transparency B': [transparency_b] * len(rewards_b),
            'Game': [game] * len(rewards_b)
        })
        save_dataframe(rewards_df_a, os.path.dirname(file), 'rewards_a', game)
        save_dataframe(rewards_df_b, os.path.dirname(file), 'rewards_b', game)

    return df

def process_metric(results_dir, metric, continuous_games, action_labels_dict):
    """Process a single metric and save to disk."""
    existing_df = load_dataframe(metric, results_dir)
    if existing_df is not None:
        logger.info(f"Loaded existing {metric} dataframe for {results_dir}")
        return existing_df

    files = [os.path.join(results_dir, file) for file in os.listdir(results_dir) if file.endswith('.json.gz')]
    dfs = Parallel(n_jobs=-2)(delayed(process_file)(file, continuous_games, metric, action_labels_dict) for file in files)
    combined_df = pd.concat(dfs, ignore_index=True)
    game = combined_df['Game'].iloc[0] if not combined_df.empty else 'Unknown'
    save_dataframe(combined_df, results_dir, metric, game)
    return combined_df

def process_results(results_dir, continuous_games, metrics, action_labels_dict):
    """Process results for all metrics."""
    for metric in metrics.keys():
        df = process_metric(results_dir, metric, continuous_games, action_labels_dict)
        logger.info(f"Processed metric {metric} for {results_dir}")

def process_dir_wrapper(results_dir, continuous_games, metrics, action_labels_dict):
    try:
        logger.info(f"Processing {results_dir}")
        process_results(results_dir, continuous_games, metrics, action_labels_dict)
        logger.info(f"Completed processing {results_dir}")
    except Exception as e:
        logger.error(f"Error processing {results_dir}: {e}")

        

results_plot = 'results_plot'
results_dir_prefix = "results/results_"
results_dirs = [f"{results_dir_prefix}{i}" for i in range(0, 8)]

if __name__ == "__main__":
    os.makedirs(results_plot, exist_ok=True)
    existing_dirs = [dir for dir in results_dirs if os.path.exists(dir)]
    logger.info(f"Processing directories: {existing_dirs}")

    continuous_games = ["BertrandCompetition"]  # Add more continuous games if needed
    action_labels_dict = {
    'GameOfChicken': {0: 'Swerve', 1: 'Straight'},
    'PrisonersDilemma': {0: 'Cooperate', 1: 'Defect'},
    'StagHunt': {0: 'Hunt Hare', 1: 'Hunt Stag'},
    'EntryDeterrenceGame': {0: 'Low Price', 1: 'High Price'},
    'StackelbergGame': {i: f'Production Level {i}' for i in range(10)},
    'UltimatumGame': {i: f'Offer {i} Units' for i in range(11)},
    'PublicGoodsGame': {i: f'Contribute {i} Units' for i in range(11)},
    'BargainingGame': {i: f'Offer {i} Units' for i in range(11)},
    'MatchingPennies': {0: 'Heads', 1: 'Tails'},
    'RockPaperScissors': {0: 'Rock', 1: 'Paper', 2: 'Scissors'},
    'BattleOfTheSexes': {0: "A's Preferred", 1: "B's Preferred"},
    'CoordinationGame': {0: 'Option 1', 1: 'Option 2'},
    'BertrandCompetition': {0: 'Price'}
    }

    if platform.system() != 'Darwin':
        Parallel(n_jobs=1)(delayed(process_dir_wrapper)(dir, continuous_games, metric_titles, action_labels_dict) for dir in existing_dirs)
    else:
        for dir in existing_dirs:
            process_dir_wrapper(dir, continuous_games, metric_titles, action_labels_dict)


In [ ]:
import pandas as pd
import os

# Function to load and display a specific metric's dataframe from a given directory
def load_and_display_metric(results_dir, metric):
    file_path = os.path.join(results_dir, f"{metric}.parquet")
    if os.path.exists(file_path):
        df = pd.read_parquet(file_path)
        return df
    else:
        print(f"File for {metric} not found in {results_dir}")
        return None

results_dir = "results/results_0"  # change to the directory you want to check
metric = "actor_losses_a"  # change to the metric you want to check

# Load and display the metric
df = load_and_display_metric(results_dir, metric)
df